In [1]:
scale_cadence = 3

In [2]:
# import pandas as pd
# from datetime import datetime,timedelta
# import numpy as np

# arc_timesfile = '/data/SatDragModelValidation/data/inputs/sat_icesat2/arc_times.txt'


# ### Make a function that finds the right range of dates in this file.
# arcs = pd.read_csv(arc_timesfile, 
#             sep = ',',
# #             dtype=object,
#             names = [
#                 'arc'         ,
#                 'epoch_start' ,
#                 'epoch_stop'  ,
#                 'orbit_start' ,
#                 'orbit_stop'  ,
#                     ],)
# arcs_yyyyddd = [x.strip() for x in arcs['arc'].values.tolist()]
# epochstart   = [x.strip() for x in arcs['orbit_start'].values.tolist()]
# epochstop    = [x.strip() for x in arcs['orbit_stop' ].values.tolist()]

import pandas as pd
from datetime import datetime,timedelta
import numpy as np

timestart = pd.to_datetime('2018-10-23 00:00:00', format='%Y-%m-%d %H:%M:%S')
timeend   = pd.to_datetime('2019-05-05 00:00:00', format='%Y-%m-%d %H:%M:%S')
scale_cadence = 3

arcs_yyyyddd         = []
epochstart = []
epochstop   = []

itime = timestart
while itime < timeend:
               
    itime_0 = itime
    itime = itime + pd.to_timedelta(24,'h')
    
    arcs_yyyyddd.append(itime_0.strftime('%Y.%j'))
    epochstart.append(itime_0.strftime('%Y-%m-%d %H:%M:%S'))
    epochstop.append(itime.strftime(  '%Y-%m-%d %H:%M:%S'))

#     print(' ',itime_0.strftime('%Y.%j'),itime_0, ' to ', itime)        


##### update the epoch start to be 3-hours before and epoch end to be 3 hours later
input_arcs       = []
input_epochstart = []
input_epochstop   = []


for i in epochstart:
    itime = pd.to_datetime(i) - pd.to_timedelta(3,'h')
    input_epochstart.append(itime.strftime('%Y-%m-%d %H:%M:%S'))

for i in epochstop:
    itime = pd.to_datetime(i) + pd.to_timedelta(3,'h')
    input_epochstop.append(itime.strftime('%Y-%m-%d %H:%M:%S'))

In [3]:
from pygeodyn.PYGEODYN import Pygeodyn
import datetime


scaling_cadence = 3
scale_cadence = scaling_cadence
index_buffarc_start = 0
run_list = [
                'jb2008',
                'msis2',
                'dtm2020_o',
                'hasdm_oc'
           ]

month_list = ['oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr']#,
# month_list = ['dec', 'jan']#,
# month_list = [ 'mar', 'apr']#,


# month_list = ['nov']#,



dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    for imonth,month in enumerate(month_list):
        if i =='msis2':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 5
            run_dict[month+i]['model_path'] = None
        if i =='dtm2020_o':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 3
            run_dict[month+i]['model_path'] = None
        if i =='jb2008':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 1
            run_dict[month+i]['model_path'] = None
        if i =='hasdm_oc':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 2 
            run_dict[month+i]['model_path'] = dir_modeldat+'/hasdm/sethasdm_orbitclouds' #HASDM_OrbitCloud_2018313.01.csv


print(run_dict)

{'octjb2008': {'num': 1, 'model_path': None}, 'novjb2008': {'num': 1, 'model_path': None}, 'decjb2008': {'num': 1, 'model_path': None}, 'janjb2008': {'num': 1, 'model_path': None}, 'febjb2008': {'num': 1, 'model_path': None}, 'marjb2008': {'num': 1, 'model_path': None}, 'aprjb2008': {'num': 1, 'model_path': None}, 'octmsis2': {'num': 5, 'model_path': None}, 'novmsis2': {'num': 5, 'model_path': None}, 'decmsis2': {'num': 5, 'model_path': None}, 'janmsis2': {'num': 5, 'model_path': None}, 'febmsis2': {'num': 5, 'model_path': None}, 'marmsis2': {'num': 5, 'model_path': None}, 'aprmsis2': {'num': 5, 'model_path': None}, 'octdtm2020_o': {'num': 3, 'model_path': None}, 'novdtm2020_o': {'num': 3, 'model_path': None}, 'decdtm2020_o': {'num': 3, 'model_path': None}, 'jandtm2020_o': {'num': 3, 'model_path': None}, 'febdtm2020_o': {'num': 3, 'model_path': None}, 'mardtm2020_o': {'num': 3, 'model_path': None}, 'aprdtm2020_o': {'num': 3, 'model_path': None}, 'octhasdm_oc': {'num': 2, 'model_path': 

In [4]:
# arcdate_for_files = '2018305'

# print(pd.to_datetime(arcdate_for_files, format= '%Y%j'))
# print()


In [5]:
%load_ext autoreload
%autoreload 2
from gc import collect as gc_collect
import pickle 
import os
from datetime import datetime,timedelta


g2b_path = "/data/SatDragModelValidation/data/inputs/sat_gracefoc/g2b/"
dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
             + 'gracefoc/cross_validation/dria_3hrscale/'

obj = {}

for imonth,month in enumerate(month_list):
    if month=='oct':
        y_num = 2018
        m_num = 10
    if month=='nov':
        y_num = 2018
        m_num = 11
    if month=='dec':
        y_num = 2018
        m_num = 12
    if month=='jan':
        y_num = 2019
        m_num = 1
    if month=='feb':
        y_num = 2019
        m_num = 2
    if month=='mar':
        y_num = 2019
        m_num = 3
    if month=='apr':
        y_num = 2019
        m_num = 4
        
    file_raw_ICs = f"{g2b_path}GRACEFOC_RawEphem_{y_num}_{m_num:02d}.txt"
    file_g2b     = f"pce_gracefoc_pso_{y_num}_{m_num:02d}"

    input_arcs       = []
    input_epochstart = []
    input_epochstop  = []
    arc_length = []
    scaleparameter_times = []
    
    for i,val in enumerate(arcs_yyyyddd):
        if pd.to_datetime(val[:8],format="%Y.%j").month == m_num:

            itime0 = pd.to_datetime(epochstart[i]) - pd.to_timedelta(3,'h')
            itime1 = pd.to_datetime(epochstop[i]) + pd.to_timedelta(3,'h')
            input_epochstart.append(itime0.strftime('%Y-%m-%d %H:%M:%S'))
            input_epochstop.append(itime1.strftime('%Y-%m-%d %H:%M:%S'))
            
            epoch_delta  =  pd.to_timedelta(pd.to_datetime(itime1) - pd.to_datetime(itime0), 'hours')
            arc_length.append(epoch_delta.total_seconds()/3600)

        ### Use Arc text file times
            input_arcs.append(      arcs_yyyyddd[i])
            scalestart = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j"))
            scalestop  = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j") + pd.to_timedelta(24,'h'))
            scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                      end   = pd.to_datetime(scalestop ),
                                                      freq=str(scale_cadence)+"h")

            ###append the epoch end time to the end
            add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                          pd.to_datetime(pd.Series(itime1)).values.astype(np.int64) // 10 ** 9)
            scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                format ='%y%m%d%H%M%S.%f' )
                             for ts in add_epochend ])

    ### Skip the days that makes the 2018-2019 transitions so hard for hasdm
    
#     if month=='dec':
#         input_arcs           = input_arcs[:-2] 
#         input_epochstart     = input_epochstart[:-2] 
#         input_epochstop      = input_epochstop[:-2] 
#         scaleparameter_times = scaleparameter_times[:-2] 

#     if month=='jan':
#         input_arcs           = input_arcs[2:] 
#         input_epochstart     = input_epochstart[2:] 
#         input_epochstop      = input_epochstop[2:] 
#         scaleparameter_times = scaleparameter_times[2:] 
    
    if month=='feb':
        #####  remove the weird day in march
        for i in np.arange(37,53):
            print(f"skipping 2019.{i:03d}")
            index = input_arcs.index(f'2019.{i:03d}')
        
            input_arcs.pop(index)
            input_epochstart.pop(index)
            input_epochstop.pop(index)
            scaleparameter_times.pop(index)       

    if month=='mar':
        #####  remove the weird day in march
        index = input_arcs.index('2019.078')
        input_arcs.pop(index)
        input_epochstart.pop(index)
        input_epochstop.pop(index)
        scaleparameter_times.pop(index)       
        
        
    for i,val in enumerate(input_arcs):
        print(f" {input_arcs[i]:10}  , {input_epochstart[i]:20},  {input_epochstop[i]:20}, {arc_length[i]}")
        if val[8:] == 'A' or val[8:] == 'B':
            print("")

        else:
            print(f" {'':10}  , {scaleparameter_times[i]}")
            print("")    
    
    for i,den in enumerate(run_list):
        

        settings_gfoC= {# Basic input settings
                     'satellite'      : {'input': 'gracefoc'},
                     'den_model'      : {'input': den},
                     'run_type'       : {'input': 'DataReduction_PCE'},
                     'run_specifier'  : {'input': '_'+month},
                     'cd_model'       : {'input': 'DRIA'},
                     'file_string'    : {'input': 'DRIAscaled_'},
                     'model_data_path' : {'input': run_dict[month+den]['model_path']},
                     # Force Model settings
                      'cd_type'               : {'input':"Fixed_CD"},
                      'cd_value'              : {'input':1.0 },
                      'scaling_factor'        : {'input':True},
                      'hours_between_cd_adj'  : {'input':scaling_cadence},
                      #### Comment for unadjusted run:
                      'cd_adjustment_boolean' : {'input':True },
                    #### DRIA CD Model Parameters
                    'cd_model_params' : {'input':{ 
                            'MS'     : '26.980D0'   ,  #!  molar mass for each panel (g/mol)
                            'TW'     : '300.0D0'    ,  #!  temperature of panels  (K)
                               ###  Alpha is b/w 0 and 1
                            'ALPHA'  : '0.890D0'    ,  #!  accomodation coefficient
                            'KL'     : '0.0D0'    ,    #!  langmuir parameter
                            'FRACOX' : '1.0D0'   ,     #!  fraction of surface covered by atomic oxygen
                       }},
                      #### ---------------------------------------
                     # Run
                      'arc_type'       : {'input':'Nominal30hr_and_AB'},      
                      'step'           : {'input': 10.},
                      'orbfil_step'    : {'input': 120.},
                      'which_ICfile'   : {'input':file_raw_ICs},
                      'which_g2bfile'  : {'input':file_g2b},
                        #
                      'arc'            : {'input': input_arcs},
                      'epoch_start'    : {'input': input_epochstart},
                      'epoch_stop'     : {'input': input_epochstop},  
                'scaleparameter_times' : {'input': scaleparameter_times},  
                       #                                
                      'global_options' : {'input':'pso_2018'},
                     # Request read on raw outputs
                      'request_data'   : {'input': ['Trajectory_orbfil', 
                                                   'Density', 
                                                   'Residuals_summary',
                                                   'DragFile',
                                                   'AdjustedParams'
                                                   ]},
                  #end dict
                  }



#         sat = Pygeodyn(settings_gfoC, use_file=False)
#         sat.run_arcs()
# #         obj[month+den] =  sat.getData_BigData_lowmemory(orbit_propagation=False)
# #         obj[month+den] = vars(obj[month+den])
#         gc_collect()


        pickleName = f'_{month}_DRIA_scale{scale_cadence}.pkl'

        pickle_file = dir_save+den+pickleName
        if not os.path.exists(pickle_file):
            print('Must create pickle file...')
            print('   ',  pickle_file)
            print('   ', 'Reading Geodyn Data')

            ### Load the data into an object
            sat = Pygeodyn(settings_gfoC, use_file=False)
            obj = sat.getData_BigData_lowmemory(orbit_propagation=False)
            gc_collect()

            #### Pickle the object to save it
            print('   ', 'Saving pickle')
            filehandler = open(pickle_file, 'wb') 
            pickle.dump(vars(obj), filehandler)
            filehandler.close()
            obj = 0
            print('   ', 'Saved pickle')

obj = {}
for i,model in enumerate(run_list):
    for imonth,month in enumerate(month_list):
        pickleName = f'_{month}_DRIA_scale{scale_cadence}.pkl'

        ### Load the data if the pickles exist
        print()
        print()
        gc_collect()

        pickle_file = dir_save+model+pickleName

        obj[month+model] = pd.read_pickle(pickle_file)
#         filehandler = open(pickle_file, 'rb') 
#         obj[month+model] = pickle.load(filehandler)
#         filehandler.close()
        print('Loaded data from pickle... ',  model)
    
### Save space if doing density retrieval
for model in run_dict.keys():
    del obj[model]['OrbitResids']
    del obj[model]['Trajectory_orbfil']
    
gc_collect()

 2018.296    , 2018-10-22 21:00:00 ,  2018-10-24 03:00:00 , 30.0
             , [Timestamp('2018-10-23 00:00:00'), Timestamp('2018-10-23 03:00:00'), Timestamp('2018-10-23 06:00:00'), Timestamp('2018-10-23 09:00:00'), Timestamp('2018-10-23 12:00:00'), Timestamp('2018-10-23 15:00:00'), Timestamp('2018-10-23 18:00:00'), Timestamp('2018-10-23 21:00:00'), Timestamp('2018-10-24 00:00:00'), Timestamp('2018-10-24 03:00:00')]

 2018.297    , 2018-10-23 21:00:00 ,  2018-10-25 03:00:00 , 30.0
             , [Timestamp('2018-10-24 00:00:00'), Timestamp('2018-10-24 03:00:00'), Timestamp('2018-10-24 06:00:00'), Timestamp('2018-10-24 09:00:00'), Timestamp('2018-10-24 12:00:00'), Timestamp('2018-10-24 15:00:00'), Timestamp('2018-10-24 18:00:00'), Timestamp('2018-10-24 21:00:00'), Timestamp('2018-10-25 00:00:00'), Timestamp('2018-10-25 03:00:00')]

 2018.298    , 2018-10-24 21:00:00 ,  2018-10-26 03:00:00 , 30.0
             , [Timestamp('2018-10-25 00:00:00'), Timestamp('2018-10-25 03:00:00'), Timesta

 2019.001    , 2018-12-31 21:00:00 ,  2019-01-02 03:00:00 , 30.0
             , [Timestamp('2019-01-01 00:00:00'), Timestamp('2019-01-01 03:00:00'), Timestamp('2019-01-01 06:00:00'), Timestamp('2019-01-01 09:00:00'), Timestamp('2019-01-01 12:00:00'), Timestamp('2019-01-01 15:00:00'), Timestamp('2019-01-01 18:00:00'), Timestamp('2019-01-01 21:00:00'), Timestamp('2019-01-02 00:00:00'), Timestamp('2019-01-02 03:00:00')]

 2019.002    , 2019-01-01 21:00:00 ,  2019-01-03 03:00:00 , 30.0
             , [Timestamp('2019-01-02 00:00:00'), Timestamp('2019-01-02 03:00:00'), Timestamp('2019-01-02 06:00:00'), Timestamp('2019-01-02 09:00:00'), Timestamp('2019-01-02 12:00:00'), Timestamp('2019-01-02 15:00:00'), Timestamp('2019-01-02 18:00:00'), Timestamp('2019-01-02 21:00:00'), Timestamp('2019-01-03 00:00:00'), Timestamp('2019-01-03 03:00:00')]

 2019.003    , 2019-01-02 21:00:00 ,  2019-01-04 03:00:00 , 30.0
             , [Timestamp('2019-01-03 00:00:00'), Timestamp('2019-01-03 03:00:00'), Timesta

skipping 2019.037
skipping 2019.038
skipping 2019.039
skipping 2019.040
skipping 2019.041
skipping 2019.042
skipping 2019.043
skipping 2019.044
skipping 2019.045
skipping 2019.046
skipping 2019.047
skipping 2019.048
skipping 2019.049
skipping 2019.050
skipping 2019.051
skipping 2019.052
 2019.032    , 2019-01-31 21:00:00 ,  2019-02-02 03:00:00 , 30.0
             , [Timestamp('2019-02-01 00:00:00'), Timestamp('2019-02-01 03:00:00'), Timestamp('2019-02-01 06:00:00'), Timestamp('2019-02-01 09:00:00'), Timestamp('2019-02-01 12:00:00'), Timestamp('2019-02-01 15:00:00'), Timestamp('2019-02-01 18:00:00'), Timestamp('2019-02-01 21:00:00'), Timestamp('2019-02-02 00:00:00'), Timestamp('2019-02-02 03:00:00')]

 2019.033    , 2019-02-01 21:00:00 ,  2019-02-03 03:00:00 , 30.0
             , [Timestamp('2019-02-02 00:00:00'), Timestamp('2019-02-02 03:00:00'), Timestamp('2019-02-02 06:00:00'), Timestamp('2019-02-02 09:00:00'), Timestamp('2019-02-02 12:00:00'), Timestamp('2019-02-02 15:00:00'), Times

 2019.091    , 2019-03-31 21:00:00 ,  2019-04-02 03:00:00 , 30.0
             , [Timestamp('2019-04-01 00:00:00'), Timestamp('2019-04-01 03:00:00'), Timestamp('2019-04-01 06:00:00'), Timestamp('2019-04-01 09:00:00'), Timestamp('2019-04-01 12:00:00'), Timestamp('2019-04-01 15:00:00'), Timestamp('2019-04-01 18:00:00'), Timestamp('2019-04-01 21:00:00'), Timestamp('2019-04-02 00:00:00'), Timestamp('2019-04-02 03:00:00')]

 2019.092    , 2019-04-01 21:00:00 ,  2019-04-03 03:00:00 , 30.0
             , [Timestamp('2019-04-02 00:00:00'), Timestamp('2019-04-02 03:00:00'), Timestamp('2019-04-02 06:00:00'), Timestamp('2019-04-02 09:00:00'), Timestamp('2019-04-02 12:00:00'), Timestamp('2019-04-02 15:00:00'), Timestamp('2019-04-02 18:00:00'), Timestamp('2019-04-02 21:00:00'), Timestamp('2019-04-03 00:00:00'), Timestamp('2019-04-03 03:00:00')]

 2019.093    , 2019-04-02 21:00:00 ,  2019-04-04 03:00:00 , 30.0
             , [Timestamp('2019-04-03 00:00:00'), Timestamp('2019-04-03 03:00:00'), Timesta

Loaded data from pickle...  jb2008


Loaded data from pickle...  jb2008


Loaded data from pickle...  jb2008


Loaded data from pickle...  jb2008


Loaded data from pickle...  jb2008


Loaded data from pickle...  jb2008


Loaded data from pickle...  msis2


Loaded data from pickle...  msis2


Loaded data from pickle...  msis2


Loaded data from pickle...  msis2


Loaded data from pickle...  msis2


Loaded data from pickle...  msis2


Loaded data from pickle...  msis2


Loaded data from pickle...  dtm2020_o


Loaded data from pickle...  dtm2020_o


Loaded data from pickle...  dtm2020_o


Loaded data from pickle...  dtm2020_o


Loaded data from pickle...  dtm2020_o


Loaded data from pickle...  dtm2020_o


Loaded data from pickle...  dtm2020_o


Loaded data from pickle...  hasdm_oc


Loaded data from pickle...  hasdm_oc


Loaded data from pickle...  hasdm_oc


Loaded data from pickle...  hasdm_oc


Loaded data from pickle...  hasdm_oc


Loaded data from pickle...  hasdm_oc


Loaded data 

0

In [6]:
input_arcs

['2019.091',
 '2019.092',
 '2019.093',
 '2019.094',
 '2019.095',
 '2019.096',
 '2019.097',
 '2019.098',
 '2019.099',
 '2019.100',
 '2019.101',
 '2019.102',
 '2019.103',
 '2019.104',
 '2019.105',
 '2019.106',
 '2019.107',
 '2019.108',
 '2019.109',
 '2019.110',
 '2019.111',
 '2019.112',
 '2019.113',
 '2019.114',
 '2019.115',
 '2019.116',
 '2019.117',
 '2019.118',
 '2019.119',
 '2019.120']

In [7]:
def plot_cleanformat_axes(fig, font_dict):
    rownum, colnum = fig._get_subplot_rows_columns()

    for i in rownum:
        if len(rownum)==1:
            L_ticklabel = True
        else:
            if i < len(rownum):
                L_ticklabel = False
            else:
                L_ticklabel = True
        fig.update_xaxes(### LINE at axis border
                          showline=True,
                          showticklabels=L_ticklabel,
    #                       tickformat= '%m/%d',
                          linecolor='black',
                          linewidth=1,
                         ### Major ticks
                          ticks='inside',
                          tickfont=font_dict,
                          mirror=True,
    #                       tickwidth=2,
    #                       ticklen=9,
                          tickcolor='grey',
    #                       tick0="2018-11-9" ,
    #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                          #### Minor Ticks
                           minor=dict(
                             dtick=86400000.0, # milliseconds in a day
                             tickwidth=1,
                             ticklen=4,
                             tickcolor='grey',
                             ticks='inside'),
                          ### GRID
                           gridcolor='gainsboro',
                           gridwidth=1,
                           layer='above traces',
                           tickangle=0,
                           row=i, col=1)
        fig.update_yaxes(showline=True,      # add line at x=0
                             showticklabels=True,
                             linecolor='black',  # line color
                             linewidth=1,        # line size
                         ticks='inside',     # ticks outside axis
                         tickfont=font_dict, # tick label font
                         mirror='allticks',  # add ticks to top/right axes
                         tickwidth=1,      # tick width
                         tickcolor='black',  # tick color
                         gridcolor='gainsboro',
                         gridwidth=1,
                         layer='above traces',
                         row=i, col=1)
    return(fig)

from pygeodyn.pygeodyn_plot_scalingfactors import *

coldict = {}
coldict['msis2']     = "#2ca02c"  # 'tab:green'
coldict['dtm2020_o'] = "#d62728"  # 'tab:red'
coldict['jb2008']    = "orange"   
coldict['hasdm_oc']  = "#1f77b4"     


In [8]:
  
satid = 2012003
wgts = {}

for model in run_dict.keys():
    wgts[model] = {}
    ScalingFactors  = []
    ScalingFactor_times = []

    for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
        epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
        hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
        frachours =(hrs/24)
        #
        if len(arc) == 9:
            maneuv_indicator = arc[8]
        else:
            maneuv_indicator = ''
        arc_type = obj[model]['global_params']['prms']['arc_type']
        if arc_type == "Nominal30hr_and_AB":
            arc_name =arc[:8]+ maneuv_indicator
        else:
            arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        ### Collect the weights for the ensemble average
        inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
        wgts[model][arc_name] = inv_rms#/sum_wgts

        iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
        for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
            if iit == 0 or iit==9:
                pass
            else:
                CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                ScalingFactor_times.append(itime)
    run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
    run_dict[model]['ScalingFactors']      = ScalingFactors
    run_dict[model]['Weight'] = wgts[model]
    

    
###### SCALE THE DENSITIES:
models_dens = {}
for monthmodel in run_dict.keys():
    print(f"---Making continuous scaled rho for {monthmodel}")
    models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)

    del obj[monthmodel]
    gc_collect()

del obj
gc_collect()
## Retrieve scaled ensemble weighted average
##     'Rho_x' denotes the ensemble weighted avg

models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, month_list, run_list)




---Making continuous scaled rho for octjb2008
---Making continuous scaled rho for novjb2008
---Making continuous scaled rho for decjb2008
---Making continuous scaled rho for janjb2008
---Making continuous scaled rho for febjb2008
---Making continuous scaled rho for marjb2008
---Making continuous scaled rho for aprjb2008
---Making continuous scaled rho for octmsis2
---Making continuous scaled rho for novmsis2
---Making continuous scaled rho for decmsis2
---Making continuous scaled rho for janmsis2
---Making continuous scaled rho for febmsis2
---Making continuous scaled rho for marmsis2
---Making continuous scaled rho for aprmsis2
---Making continuous scaled rho for octdtm2020_o
---Making continuous scaled rho for novdtm2020_o
---Making continuous scaled rho for decdtm2020_o
---Making continuous scaled rho for jandtm2020_o
---Making continuous scaled rho for febdtm2020_o
---Making continuous scaled rho for mardtm2020_o
---Making continuous scaled rho for aprdtm2020_o
---Making continuous

In [16]:
%load_ext autoreload
%autoreload 2


def PLOT_resids(fig,  den_dict, model_dict, _model_name_):
    
    SHOW_alldata = True
    
    model_m1 = _model_name_[3:]
#     print('model_m1', model_m1)

    fig.add_trace(go.Scattergl(x=pd.to_datetime(model_dict['ScalingFactor_times'])-pd.to_timedelta(scale_cadence/2, 'h'),
                           y=model_dict['ScalingFactors'],
                           name= model,
                           mode='markers',
                           opacity=1,
                           marker=dict(color=coldict[model_m1], size=5),
#                                marker=dict(color=coldict[model_m1], 
#                                            size=6,
#                                            symbol='line-ew',
#                                            line = dict(color = coldict[model_m1], width=3)), #symbol='diamond-wide'),
#                            line = dict(shape = 'hvh',dash ='dash', color = coldict[model_m1], width=1),
                           showlegend=False),
                           secondary_y=False,row=1, col=1)

    
    
    time_avg,dscale_avg = orbit_avg_generic(den_dict['dates'], den_dict['denscaled'], den_dict['lat'])
    ## -----------------------------------------------------------------------------------------------------
    ##     Orbit Averaged Density
    fig.add_trace(go.Scattergl(x=time_avg,
                               y=dscale_avg,
                               ### name= model_m1,
                               mode='markers+lines',
                               opacity=1,
                                   marker=dict(color=coldict[model_m1],size=2),
                                   line = dict(dash ='solid', color = coldict[model_m1], width=2),
                               showlegend=False), row=3, col=1)
#     fig.add_trace(go.Scattergl(x=den_dict['dates'],
#                                y=den_dict['denscaled'],
#                                ### name= model_m1,
#                                mode='markers+lines',
#                                opacity=1,
#                                    marker=dict(color=coldict[model_m1],size=2),
#                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                                showlegend=False), row=3, col=1)

    
    
    
    
    time_avg,d_avg = orbit_avg_generic(den_dict['dates'], den_dict['dens'], den_dict['lat'])
    ### -----------------------------------------------------------------------------------------------------
    ###     Orbit Averaged Density
#     fig.add_trace(go.Scattergl(x=den_dict['dates'], #time_avg,
#                                y=den_dict['dens'], #d_avg,
#                                ### name= model_m1,
#                                mode='markers+lines',
#                                opacity=1,
#                                    marker=dict(color=coldict[model_m1],size=1),
#                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                                showlegend=False), row=2, col=1)

    ###     Orbit Averaged Density
    fig.add_trace(go.Scattergl(x=time_avg,
                               y=d_avg,
                               ### name= model_m1,
                               mode='markers+lines',
                               opacity=1,
                                   marker=dict(color= coldict[model_m1],size=2),
                                   line = dict(dash ='solid', color = coldict[model_m1], width=2),
                               showlegend=False), row=2, col=1)
    
    
#                 time_avg,CD_avg, area_avg_rolling, CD_std = orb_avg_param(obj_m1.__dict__['DragFile'], arc_string, 'TOTAREA')
#                 fig.add_trace(go.Scattergl(x=time_avg,
#                                            y=area_avg_rolling,
#     #                                          name= plot_name+arc,
#                                              mode='markers+lines',
#                                              opacity=1,
#                                              marker=dict(color=col,size=2,),
#                                              line = dict(shape='hvh', dash ='solid', color = col, width=3),
#                                              showlegend=False, ),
#                                              secondary_y=False,
#                                              row=3, col=1,
#                                          )



        

        
    return(fig)







fig  = make_subplots(
    rows=3, cols=1,
    vertical_spacing = 0.05,
    shared_xaxes=True)


for model in run_dict.keys():
#     for imonth,month in enumerate(month_list):
    print('model', model)

    fig = PLOT_resids(fig, models_dens[model], run_dict[model], model)

    
#######################################################
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = True
    else:
        if i < len(rownum):
            L_ticklabel = True
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)

    fig.update_yaxes(title_text="Scaling Factor", 
                     exponentformat= 'power',row=1, col=1)

    fig.update_yaxes(title_text="Density", 
                     type="log", 
                     exponentformat= 'power',row=2, col=1)
    fig.update_yaxes(title_text="Scaled Density", 
                      type="log", 
                     exponentformat= 'power',row=3, col=1)

for month in month_list:

    time_avg,Rho_x = orbit_avg_generic(models_dens[month+'Rho_x']['date'],
                                            models_dens[month+'Rho_x']['Rho_x'],
                                            models_dens[month+'Rho_x']['lat'])

#     time_avg,Rho_std = orbit_avg_generic(models_dens[month+'Rho_x']['date'],
#                                             models_dens[month+'Rho_x']['Rho_std'],
#                                             models_dens[month+'Rho_x']['lat'])


    fig.add_trace(go.Scattergl(x=time_avg,
                               y=Rho_x,
                               ### name= model_m1,
                               mode='markers+lines',
                               opacity=1,
                               marker=dict(color='black',size=5),
                                   line = dict( dash ='solid', color = 'black', width=4),
                               showlegend=False), row=3, col=1)
#     ## ERRROR BARS
#     fig.add_trace(go.Scattergl(x=time_avg,
#                                y=np.array(Rho_x)\
#                                    +np.array(Rho_std),
#                                ### name= model_m1,
#                                mode='lines',
#                                opacity=.8,
#                                    line = dict( dash ='dash', color = 'grey', width=3),
#                                showlegend=False), row=3, col=1)
#     fig.add_trace(go.Scattergl(x=time_avg,
#                                y=np.array(Rho_x)\
#                                   -np.array(Rho_std),
#                                ### name= model_m1,
#                                mode='lines',
#                                opacity=.8,
#                                    line = dict( dash ='dash', color = 'grey', width=3),
#                                showlegend=False), row=3, col=1)

    

    
a='input'
s=settings_gfoC
fig.update_layout(title=f"{s['cd_model'][a]}, {s['hours_between_cd_adj'][a]}-hr Scale from CD={s['cd_value'][a]}", 
                  autosize=False,    width=850,    height=900,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=16)  # Increase size of subplot title
# fig.show(config=config)
fig.show(config= dict({
                'displayModeBar': False,
                'responsive': True,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                }),
#          renderer='jpg',
)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
model octjb2008
model novjb2008
model decjb2008
model janjb2008
model febjb2008
model marjb2008
model aprjb2008
model octmsis2
model novmsis2
model decmsis2
model janmsis2
model febmsis2
model marmsis2
model aprmsis2
model octdtm2020_o
model novdtm2020_o
model decdtm2020_o
model jandtm2020_o
model febdtm2020_o
model mardtm2020_o
model aprdtm2020_o
model octhasdm_oc
model novhasdm_oc
model dechasdm_oc
model janhasdm_oc
model febhasdm_oc
model marhasdm_oc
model aprhasdm_oc


In [10]:
# obj['mardtm2020_o']['OrbitResids']['2019.078'].keys()


In [11]:
# obj['mardtm2020_o']['OrbitResids']['2019.078']['resids'].keys()


### NTW residuals

In [12]:

# fig  = make_subplots(
#     rows=3, cols=1,
#     vertical_spacing = 0.05,
#     shared_xaxes=True)

# for ii,arc in enumerate(obj['mardtm2020_o']['global_params']['arc_input']):
#     epochstart = obj['mardtm2020_o']['global_params']['prms']['epoch_start'][ii]
#     hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#     frachours =(hrs/24)
#     #
#     if len(arc) == 9:
#         maneuv_indicator = arc[8]
#     else:
#         maneuv_indicator = ''
#     arc_type = obj['mardtm2020_o']['global_params']['prms']['arc_type']
#     if arc_type == "Nominal30hr_and_AB":
#         arc_name =arc[:8]+ maneuv_indicator
#     else:
#         arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
#     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['Date'],
#                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['N'],
#                                mode='markers+lines',
#                                opacity=1,
#                                marker=dict(color='blue', size=3),
#                                    showlegend=False),
#                                    secondary_y=False,row=1, col=1)
# #     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['Date'],
# #                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['N'],
# #                            mode='markers',
# #                            opacity=1,
# #                                marker=dict(color='red', size=3),
# #                                showlegend=False),
# #                                secondary_y=False,row=1, col=1)
    
#     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['Date'],
#                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['T'],
#                                mode='markers+lines',
#                                opacity=1,
#                                marker=dict(color='blue', size=3),
#                                    showlegend=False),
#                                    secondary_y=False,row=2, col=1)
# #     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['Date'],
# #                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['T'],
# #                            mode='markers',
# #                            opacity=1,
# #                                marker=dict(color='red', size=3),
# #                                showlegend=False),
# #                                secondary_y=False,row=2, col=1)

#     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['Date'],
#                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['W'],
#                                mode='markers+lines',
#                                opacity=1,
#                                marker=dict(color='blue', size=3),
#                                    showlegend=False),
#                                    secondary_y=False,row=3, col=1)
# #     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['Date'],
# #                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['W'],
# #                            mode='markers',
# #                            opacity=1,
# #                                marker=dict(color='red', size=3),
# #                                showlegend=False),
# #                                secondary_y=False,row=3, col=1)  
    
    
    
    
# fig.show(config= dict({
#                 'displayModeBar': False,
#                 'responsive': False,
#                 'staticPlot': True,
#                 'displaylogo': False,
#                 'showTips': False,
#                 }),
#         )



### SF and CD*A

In [13]:
# %load_ext autoreload
# %autoreload 2


# def PLOT_retrievedRHO(fig, obj_m1, den_dict, model_dict, model_name):
        
#     obj_m1=obj_m1[model_name]
# #     print(model_name)
#     model_m1 = model_name[3:]#obj_m1['global_params']['prms']['den_model']

#     fig.add_trace(go.Scattergl(x=pd.to_datetime(model_dict['ScalingFactor_times'])-pd.to_timedelta(scale_cadence/2, 'h'),
#                                y=model_dict['ScalingFactors'],
#                                name= model,
#                                mode='markers',
#                                opacity=1,
# #                                    marker=dict(color=coldict[model_m1], 
# #                                                size=5,
# #                                                symbol='line-ew',
# #                                                line = dict(color = coldict[model_m1], width=3)), #symbol='diamond-wide'),
#                                    marker=dict(color=coldict[model_m1], size=5),
#                                line = dict(shape = 'hvh',dash ='dot', color = coldict[model_m1], width=1),
#                                showlegend=False),
#                                secondary_y=False,row=1, col=1)

#     for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
#         epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
#         hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#         frachours =(hrs/24)
#         #
#         if len(arc) == 9:
#             maneuv_indicator = arc[8]
#         else:
#             maneuv_indicator = ''
#         arc_type = obj_m1['global_params']['prms']['arc_type']
#         if arc_type == "Nominal30hr_and_AB":
#             arc_name =arc[:8]+ maneuv_indicator
#         else:
#             arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        
#         drag_data        = obj_m1['DragFile'][arc_name]
#         drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
#         den_df           = obj_m1['Density' ][arc_name]
        
# #         print(arc[:8])
#         ### Cut off the extra time for the regular days
#         start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
#         end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm')

# #         print(f" {arc_name:10}|   , {start_arc},  {end_arc}")
        
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         drag_data = drag_data.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )

#         indplot = 4
#         fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                y=drag_data['CD'][::indplot]*drag_data['TOTAREA'][::indplot],
#                              mode='markers',
#                              opacity=0.3,
#                              marker=dict(color= coldict[model_m1],size=2,),
#                              showlegend=False,),
#                              secondary_y=False,
#                              row=2, col=1,
#                          )
    
#         (time_avg, avg_area) = orbit_avg_generic(drag_data['Date'],drag_data['TOTAREA'],drag_data['Lat'])    
#         (time_avg, avg_cd ) = orbit_avg_generic(drag_data['Date'],drag_data['CD'],drag_data['Lat'])    

#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_cd*avg_area,
# #                                  y=avg_cd,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=2, col=1)
    
    
             
#     return(fig)







# fig  = make_subplots(
#     rows=2, cols=1,
#     vertical_spacing = 0.05,
#     shared_xaxes=True)



# # run_dict = { }
# # rms_total_return = {}

# for model in run_dict.keys():
#     print(model)
# #     for imonth,month in enumerate(month_list):

# #     fig = PLOT_retrievedRHO(fig, obj[model],   models_dens[model], run_dict[model], model)
#     fig = PLOT_retrievedRHO(fig, obj,   models_dens[model], run_dict[model], model)


        

# # time_avg,Rho_x = orbit_avg_generic(models_dens['novRho_x']['date'],
# #                                         models_dens['novRho_x']['Rho_x'],
# #                                         models_dens['novRho_x']['lat'])

# # time_avg,Rho_std = orbit_avg_generic(models_dens['novRho_x']['date'],
# #                                         models_dens['novRho_x']['Rho_std'],
# #                                         models_dens['novRho_x']['lat'])


# # fig.add_trace(go.Scattergl(x=time_avg,
# #                            y=Rho_x,
# #                            ### name= model_m1,
# #                            mode='markers+lines',
# #                            opacity=1,
# #                            marker=dict(color='black',size=5),
# #                                line = dict( dash ='solid', color = 'black', width=4),
# #                            showlegend=False), row=2, col=1)
# # ## ERRROR BARS
# # fig.add_trace(go.Scattergl(x=time_avg,
# #                            y=np.array(Rho_x)\
# #                                +np.array(Rho_std),
# #                            ### name= model_m1,
# #                            mode='lines',
# #                            opacity=.8,
# #                                line = dict( dash ='dash', color = 'grey', width=3),
# #                            showlegend=False), row=2, col=1)
# # fig.add_trace(go.Scattergl(x=time_avg,
# #                            y=np.array(Rho_x)\
# #                               -np.array(Rho_std),
# #                            ### name= model_m1,
# #                            mode='lines',
# #                            opacity=.8,
# #                                line = dict( dash ='dash', color = 'grey', width=3),
# #                            showlegend=False), row=2, col=1)

    
    
# #######################################################
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = True
#     else:
#         if i < len(rownum):
#             L_ticklabel = True
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)

#     fig.update_yaxes(title_text="Scaling Factor", 
#                      exponentformat= 'power',row=1, col=1)

# #     yaxis_range = [-13.7 -.55 ,  -12.6+.25]# ] #full_fig.layout.yaxis2.range

# #     fig.update_yaxes(title_text="Scaled Density", 
# #                      type="log", 
# # #                       range=yaxis_range,
# #                      exponentformat= 'power',row=2, col=1)
#     fig.update_yaxes(title_text="CD*A", 
# #                       type="log", 
# #                       range=yaxis_range,
#                      exponentformat= 'power',row=2, col=1)
    
    
# fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)

# # a='input'
# # s=settings_icesat2

# fig.update_layout(#title=f"{s['cd_model'][a]}, {s['hours_between_cd_adj'][a]}-hr Scale from CD={s['cd_value'][a]}",
# #                     title=f"BWDRAG, 3-hr Scale from CD=2.5 ",
#                   autosize=False,    width=900,    height=550,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict,
#                   plot_bgcolor='white', 
#                  )
# fig.update_annotations(font_size=16)  # Increase size of subplot title
# fig.show(config= dict({
#                 'displayModeBar': False,
#                 'responsive': True,
#                 'staticPlot': False,
#                 'displaylogo': False,
#                 'showTips': False,
#                 }),
# #          renderer='jpg',
# )


# # pio.write_image(fig, 'ScalingFactors_ScaledRho.jpg', scale=5)




### ful drag state

In [14]:
# %load_ext autoreload
# %autoreload 2


# def PLOT__SATDRAG_STATE(fig, obj_m1, den_dict):
    
#     SHOW_alldata = True
    
#     model_m1 = obj_m1['global_params']['prms']['den_model']
#     if model!='hasdm_oc':
#         opac_val = 1
#     else:
#         opac_val = 1

#     for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
#         epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
#         hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#         frachours =(hrs/24)
#         #
#         if len(arc) == 9:
#             maneuv_indicator = arc[8]
#         else:
#             maneuv_indicator = ''
#         arc_type = obj_m1['global_params']['prms']['arc_type']
#         if arc_type == "Nominal30hr_and_AB":
#             arc_name =arc[:8]+ maneuv_indicator
#         else:
#             arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        



#         drag_data        = obj_m1['DragFile'][arc_name]
#         drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
#         den_df           = obj_m1['Density' ][arc_name]
        
# #         print(arc[:8])
#         ### Cut off the extra time for the regular days
#         start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
#         end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm')

# #         print(f" {arc_name:10}|   , {start_arc},  {end_arc}")
        
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         drag_data = drag_data.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         den_df = den_df.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )


#         indplot=3

# #         date_rms.append(pd.to_datetime(datetime.datetime(int(arc.split('.')[0]), 1, 1) + datetime.timedelta(int(arc.split('.')[1]))- datetime.timedelta(hours=12) ))
# #         rms_totals.append(obj_m1['Statistics'][arc]['T_RMS'].values[0])

# #         iters = obj_m1['run_parameters'+arc_name]['total_iterations']
# #         for itime in obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD']:
# #             date_scalefactor.append(itime)
# #             ScalingFactor.append(obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD'][itime]['CURRENT_VALUE'])


#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['CD'][::indplot],
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=2, col=1,
#                                  )
#         (time_avg, avg_cd ) = orbit_avg_generic(drag_data['Date'],drag_data['CD'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_cd,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=2, col=1)

#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['TOTAREA'][::indplot],
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=3, col=1,
#                                  )
#         (time_avg, avg_area) = orbit_avg_generic(drag_data['Date'],drag_data['TOTAREA'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_area,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=3, col=1)

#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['CD'][::indplot]*drag_data['TOTAREA'][::indplot],
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=4, col=1,
#                                  )
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_cd*avg_area,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=4, col=1)

        
#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['VELREL'][::indplot],
# #                                          name= plot_name+arc,
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      line = dict(shape='hvh', dash ='solid', color = coldict[model_m1], width=2),
#                                      showlegend=False, ),
#                                      secondary_y=False,
#                                      row=5, col=1,
#                                  )
#         (time_avg, avg ) = orbit_avg_generic(drag_data['Date'],drag_data['VELREL'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=5, col=1)

#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                    y=drag_data['SpeedRatio'][::indplot],
# #                                          name= plot_name+arc,
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      line = dict(shape='hvh', dash ='solid', color = coldict[model_m1], width=2),
#                                      showlegend=False, ),
#                                      secondary_y=False,
#                                      row=6, col=1,
#                                  )
#         (time_avg, avg ) = orbit_avg_generic(drag_data['Date'],drag_data['SpeedRatio'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=6, col=1)
#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=den_df['Date'][::indplot],
#                                  y=den_df['rho (kg/m**3)'][::indplot],
#                        ### name= model_m1,
#                        mode='markers',
#                        opacity=0.5,
#                            marker=dict(color=coldict[model_m1],size=2),
#                            line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                        showlegend=False), row=1, col=1)
        
#         (time_avg, d_avg ) = orbit_avg_generic(den_df['Date'],den_df['rho (kg/m**3)'],
#                                                den_df['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=d_avg,
#                            mode='lines',
#                            opacity=1,
#                                marker=dict(color=coldict[model_m1],size=1),
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=1, col=1)

    

    
    
# #     (time_avg, d_avg ) = orbit_avg_generic(den_dict['dates'],
# #                                    den_dict['den'],
# #                                    den_dict['lat'])
# #     ### -----------------------------------------------------------------------------------------------------
# #     ###     Orbit Averaged Density
# #     fig.add_trace(go.Scattergl(x=time_avg,
# #                                y=d_avg,
# #                                ### name= model_m1,
# #                                mode='markers',
# #                                opacity=opac_val,
# #                                    marker=dict(color= coldict[model_m1],size=2),
# #                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
# #                                showlegend=False), row=1, col=1)
    
    
# #                 time_avg,CD_avg, area_avg_rolling, CD_std = orb_avg_param(obj_m1.__dict__['DragFile'], arc_string, 'TOTAREA')
# #                 fig.add_trace(go.Scattergl(x=time_avg,
# #                                            y=area_avg_rolling,
# #     #                                          name= plot_name+arc,
# #                                              mode='markers+lines',
# #                                              opacity=1,
# #                                              marker=dict(color=col,size=2,),
# #                                              line = dict(shape='hvh', dash ='solid', color = col, width=3),
# #                                              showlegend=False, ),
# #                                              secondary_y=False,
# #                                              row=3, col=1,
# #                                          )



        

        
#     return(fig)







# fig  = make_subplots(
#     rows=6, cols=1,
# #     subplot_titles=(['Density (kg/m^3)'   , 
# # #                      'CD*A'               ,
# #                      'CD'                 ,   
# #                      'Relative Velocity'  , 
# #                      'Speed Ratio',
# #                     ]),
#     vertical_spacing = 0.05,
# #     horizontal_spacing = 0.05,
#     shared_xaxes=True)



# # run_dict = { }
# # rms_total_return = {}

# for model in run_dict.keys():
# #     for imonth,month in enumerate(month_list):

# #     for ialph,alpha in enumerate(alphas):   
#          # fig,rms_total_return = PLOT__intrack_residuals_w_rms_w_cd(fig, Obj_Geodyn['msis2'] , 0,  arc_listlist)
# #         run_dict[val+"alpha_"+alpha] = ialph

#     fig = PLOT__SATDRAG_STATE(fig, obj[model],   models_dens[model])

    
# #######################################################
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = True
#     else:
#         if i < len(rownum):
#             L_ticklabel = True
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)

    
#     fig.update_yaxes(title_text="Density", 
#                      type="log", 
#                      exponentformat= 'power',row=1, col=1)
#     fig.update_yaxes(title_text="CD", 
#                      exponentformat= 'power',row=2, col=1)
#     fig.update_yaxes(title_text="Total Area", 
#                      exponentformat= 'power',row=3, col=1)
#     fig.update_yaxes(title_text="CD*Area", 
#                      exponentformat= 'power',row=4, col=1)

#     fig.update_yaxes(title_text="Rel. Velocity [m/s] ", 
#                      exponentformat= 'power',row=5, col=1)
#     fig.update_yaxes(title_text="Speed Ratio", 
#                      exponentformat= 'power',row=6, col=1)

#     ###
#     ###  DATE on Final x-Axis only
# #     fig.update_xaxes(title="Date",
# #                      row=4, col=1)
# #     fig.update_xaxes(title="Date",
# #                      row=5, col=2)




# fig.update_layout(
# #                   title = '',
#                   autosize=False,    width=1000,    height=1100,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict,
#                   plot_bgcolor='white', 
#                  )
# fig.update_annotations(font_size=16)  # Increase size of subplot title



# # fig.add_vrect(x0="2018-10-16 23:00:00" ,
# #               x1="2018-10-17 00:24:00" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="2018-10-28 16:00:00" ,
# #               x1="2018-10-28 17:36:00" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="2018-10-29 10:18:00" ,
# #               x1="2018-10-29 11:42:00" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="" ,
# #               x1="" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="" ,
# #               x1="" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)

# fig.show(config=config)


